# Desktop DNS Notebook 2

## 2.1 Checking the current case

### 2.1.1 Reading in the case files
First let's start with reading in the current case and viewing the flow.

In [2]:
# reload modules
%reload_ext autoreload
%autoreload 2

# pop-out figure
%matplotlib qt

In [3]:
from postpro.read_flo import *
from postpro.plot_flo import *

# case name
casename = 'notebook1'

# read flo
prop,geom=read_flo(casename)






gpu input file found
2


Plot the flow

In [4]:

#plt = plot_flo(prop,geom,'vortz',[-1000,1000])

plt = plot_flo(prop,geom,'p',[0.96e5,1.0e5])
#plt = plot_flo(prop,geom,'po',[0.96e5,1.0e5])



Plot the loading

In [15]:
import matplotlib.pyplot as plt

wall_blocks = [3,4,5,7]

plt.figure(1)
for i in wall_blocks:
    plt.plot(geom[i-1]['x'][:,-1],prop[i-1]['p'][:,-1,0],'-k.')

plt.show()


Plot time history

In [61]:
## Plot the monitor file

from postpro.plot_monitor import *

plot_monitor(casename);

gpu input file found
2


In [17]:

plot_flo(prop,geom,'mach',[0,0.3])

In [19]:
from postpro.boundarylayer import *


boundarylayer(casename)


gpu input file found
2
gpu input file found
2


## Running in 3D

In [21]:
from meshing.write_flo import *
from meshing.write_case import *

casename_old = 'notebook1'
casename_new = 'notebook2'

# read casefiles and flow from old directory
case = read_case(casename_old)
prop,_=read_flo(casename_old)

# copy files to new directory
case['casename'] = casename_new 
write_case(case)

# write flow files to new directory
write_flo(prop,casename_new)

# set number of spanwise point
case['solver']['nk'] = 15 # spanwise points for 3D calculation 
case['solver']['span'] = 0.1 # set the spanwise extent (in this case the axial chord is 1.0m, so span=10%cax)


# adjust iterations 
case['solver']['niter'] = 10000 # total number of iterations
case['solver']['irestart'] = 12 # restart the calculation with irestart=12, which extrudes the 2D flow
case['casename'] = casename_new # copy files to new directory

# write new case files
write_case(case)




gpu input file found
2
gpu input file found
2
Writing inputs for 3dns_gpu
Total ij points
73696
Total GPUs
1
gpu input file found
2
Writing inputs for 3dns_gpu
Total ij points
73696
Total GPUs
1


## Re-start the 3D calcs

In [35]:

casename = 'notebook2'

# read casefiles
case = read_case(casename)

# adjust iterations 
case['solver']['niter'] = 10000 # total number of iterations
case['solver']['irestart'] = 1 # restart the 3D calculation 

# write new case files
write_case(case)

gpu input file found
2
Writing inputs for 3dns_gpu
Total ij points
73696
Total GPUs
1


In [5]:

# read flo
prop,geom=read_flo(casename)

gpu input file found
2


In [8]:

plt = plot_flo(prop,geom,'w',[-5,5])

## Adding inflow turbulence 




In [38]:
from meshing.read_case import *
from meshing.write_case import *

# read casefiles
case = read_case(casename)

# adjust iterations 
case['solver']['niter'] = 100000 # total number of iterations
case['solver']['irestart'] = 1 # restart the calculation with irestart=1

# add inlet perturbations
case['bcs']['aturb'] = 50.0 # amplification factor for turbulence
case['bcs']['ilength'] = 500 # inlet turbulence generation frequency (no. steps)

# write new case files
write_case(case)


gpu input file found
2
Writing inputs for 3dns_gpu
Total ij points
73696
Total GPUs
1



### 2.1.2 Creating probe files
Probe files are very useful for monitoring the flow development and convergence. The script below sets probes which are written by the solver.

In [39]:
from meshing.write_probe import *

probe = {}
probe['nprobe'] = 3 # 3 probes
probe['nwrite'] = 10 # store every 10 iterations

# probe 1 - in inlet
probe[0] = {}
probe[0]['block'] = 1 # block number 
probe[0]['i'] = 100 # i value
probe[0]['j'] = 50 # j value
probe[0]['k'] = 1  # k value

# probe 2 - in exit
probe[1] = {}
probe[1]['block'] = 8
probe[1]['i'] = 100
probe[1]['j'] = 50
probe[1]['k'] = 1

# probe 3 - near suction surface
probe[2] = {}
probe[2]['block'] = 5
probe[2]['i'] = 100
probe[2]['j'] = 20
probe[2]['k'] = 1

# write probe.txt file
write_probe(casename,probe)


### 2.1.3 Reading and plotting probe files
Now every time you run the solver the probe data will be written out into files called 'probe_#' where # is the probe number (in this case 1 and 2). The script below can be used to plot the probe data as a time series.


In [62]:
#import matplotlib.pyplot as plt
from postpro.read_probe import *

probe = read_probe(casename)

plt.figure(1)
for i in range(len(probe)):
    # plot the probe data
    print('Probe ' + str(i+1) + ' position')
    print('x = ', probe[i]['x'])
    print('y = ', probe[i]['y'])
    
    plt.subplot(1,3,i+1)
    plt.plot(probe[i]['time'],probe[i]['vel'],'-k')
    plt.title('probe ' + str(i+1))
    plt.xlabel('Time (s)')
    plt.ylabel('velocity (m/s)')
    
plt.show()



gpu input file found
2
Probe 1 position
x =  -0.39424704393697757
y =  0.06488748686689526
Probe 2 position
x =  1.739573584997901
y =  0.8455891840318108
Probe 3 position
x =  0.38613911521680955
y =  0.3951722365638744


In [9]:
from postpro.read_flo import *
from postpro.plot_flo import *

casename = 'notebook2'

# read flo
prop,geom=read_flo(casename)

gpu input file found
2


In [10]:
plot_flo(prop,geom,'w',[-5,5])

<module 'matplotlib.pyplot' from '/home/aw329/.local/lib/python3.6/site-packages/matplotlib/pyplot.py'>

## Gathering statistics

In [50]:

# read casefiles
case = read_case(casename)

# adjust iterations 
case['solver']['niter'] = 100000 # total number of iterations
case['solver']['irestart'] = 1 # restart the calculation

case['solver']['istats'] = 2 # switch-on time-averaging (istats=2 does a spanwise average as well for 2D stats)

# write new case files
write_case(case)

gpu input file found
2
Writing inputs for 3dns_gpu
Total ij points
73696
Total GPUs
1


### Read the mean flow

In [5]:
from postpro.read_2d_mean import *
from postpro.plot_flo import *

casename = 'notebook2'

prop,geom=read_2d_mean(casename,[1])




[0.10262726] 0.102627255 [2]
gpu input file found
2


In [6]:

plot_flo(prop,geom,'mach',[0,0.3])
#plot_flo(prop,geom,'po',[0.96e5,1.01e5])


<module 'matplotlib.pyplot' from '/home/aw329/.local/lib/python3.6/site-packages/matplotlib/pyplot.py'>

### Plot the loading

In [23]:
wall_blocks = [3,4,5,7]

plt.figure(1)
for i in wall_blocks:
    plt.plot(geom[i-1]['x'][:,-1],prop[i-1]['p'][:,-1],'k.')

plt.show()

### Check resolution

In [65]:
from postpro.boundarylayer import *


bl =boundarylayer(casename);


gpu input file found
2
gpu input file found
2


In [66]:
plt.figure(1)
plt.plot(bl['ss']['x'],bl['ss']['xplus'],'-k.')
plt.plot(bl['ss']['x'],bl['ss']['yplus'],'-b.')
plt.plot(bl['ss']['x'],bl['ss']['zplus'],'-r.')

plt.show()

## Check spectrum

In [39]:
from postpro.plot_spectrum import *


plot_spectrum(casename,[1,2],20);

gpu input file found
2


## 2.3 Running in 3D
### 2.3.1 Set-up the initial case files

To restart a 3D calculation from a 2D calculation, set irestart=12. This extrudes the 2D solution in the spanwise direction. You also need to set the span and number of spanwise points. Typically set a span of around 10-20 percent of chord.


### 2.3.2 Check the initial 3D flow

### 2.3.3 Re-start the 3D flow

## 2.4 Setting-up the inflow turbulence

### 2.4.1 Creating the inflow turbulence file
In this section we create the inflow turbulence file using the 'turbid' program. This is based on the method of Philips and Fyfe. The code produces filtered random numbers using a gaussian filter to give spatial correlation to the values. The filter width and length scale sets the spectrum - increasing the filter width gives a larger range of scales, while increasing the length scales shifts the spectrum to lower frequencies (larger wavelengths). 




Navigate to the case folder

> cd notebook2

Make sure the turbulence generator 'turbid' is in the case folder or on your path.

Now run the turbulence generator 

> ./turbid

### 2.4.2 Running with inflow turbulence

Once turbid has finished, you should see a file call 'inflow_turb.dat' in the case directory. This file holds the velocity perturbations that will be imposed at the inlet to the domain.

The perturbations are normalized to have a standard deviation of 1.0. This means that you need to scale the value of the parameter bcs['aturb'] to achieve a desired turbulence intensity- the perturbations are multiplied by this number during the solution. 

It is important to note that the inlet boundary condition method applies a characteristic condition over the first 20 i-points in the inlet domain. 

Within this buffer region the amplitude of the incoming waves is set to impose the turbulence fluctuations. The effect of this buffer region means that in this region the flow does not obey the true Navier-Stokes behaviour and so these regions should not be included in any post-processing (such as to compute loss).

It is also important to determine the actual spectra imposed by computing the statistics downstream of this region. This can be done with the probe data and also the 3D flow field. 

Once the inflow turbulence file has been generated, modify the case files (as shown below) and re-start the calculation.



It is important to note:
* The size of the inflow turbulence file must match the inlet dimensions in the j-k plane. Therefore if the grid is modified, a new inflow turbulence file will be needed. The script above can be used for this.
* $l_{turb}$ is the axial spacing of inflow turbulence slices- typically set to ~ (pitch) / (total pitchwise points) 
* The inflow turbulence file is read in with a freqency $f=(u_{ref}/l_{turb})$, where $u_{ref}$ is the axial velocity at inlet $u_{ref}=v_{in} \cos(\alpha_{in})$.  
* ilength sets the number of axial slices to read in before the turbulence file is rewound to the start. Setting short lengths will reduce the inflow turbulence file but give poor spectra. Typically set this to around 500.


The values in the inflow turbulence file are not a solution to the Navier-Stokes equations and so there will be some development length required for the flow to become truly turbulent. Furthermore, the nature of the characteristic boundary condition applied in the inlet region means that the initial development of the flow close to the inlet should be treated as being to some extent 'non-physical'.

An accurate determination of the inflow spectrum must therefore be computed using data some distance downstream of the inlet domain (typically > 50 points from the inlet).